In [1]:
import torch
torch.__version__

'1.7.1+cu110'

In [25]:
import pandas as pd
import csv
# 读取CSV文件
df = pd.read_csv('/mnt/mydisk/beijing.csv')

# 在第三列添加label属性并将其值设为1
df.insert(3, 'label', 1)

# 将修改后的数据写入CSV文件
df.to_csv('output.csv', index=False)





In [35]:
from datasets import load_dataset

# 加载数据集
dataset = load_dataset('csv', data_files='/mnt/mydisk/CSTD/output.csv')

# 打印数据集大小和第一个样本
print(f"Dataset size: {len(dataset['train'])}")
print(dataset['train'][0])

# 构建数据集类
# import torch

# class MyDataset(torch.utils.data.Dataset):
#     def __init__(self, dataset):
#         self.dataset = dataset

#     def __len__(self):
#         return len(self.dataset)

#     def __getitem__(self, idx):
#         sample = self.dataset[idx]
#         text = sample['text']
#         label = sample['label']
#         return text, label

# # 创建数据集对象
# my_dataset = MyDataset(dataset['train'])

# # 创建数据加载器
# data_loader = torch.utils.data.DataLoader(my_dataset, batch_size=32, shuffle=True)

# # 遍历数据加载器
# for batch in data_loader:
#     texts, labels = batch
#     print(texts, labels)

Using custom data configuration default-d44e73334a9b3470
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-d44e73334a9b3470/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset size: 24
{'title': '尹力在学习贯彻习近平新时代中国特色社会主义思想主题教育专题党课上强调 推动习近平新时代中国特色社会主义思想在京华大地落地生根形成生动实践 车俊殷勇李秀领魏小东刘伟李小三参加', 'url': 'http://whlyj.beijing.gov.cn/zwgk/xwzx/szfdt//202305/t20230509_3090538.html', 'text': '当前，全市学习贯彻习近平新时代中国特色社会主义思想主题教育正深入开展。5月8日上午，市委书记尹力围绕“为什么学”“学什么”“怎么学”“怎么干”等四个方面，在市委党校为党员干部讲了一堂主题教育专题党课。他强调，要牢牢把握“学思想、强党性、重实践、建新功”的总要求，集中精力抓好理论学习、调查研究、推动发展、检视整改，推动习近平新时代中国特色社会主义思想在京华大地落地生根，形成生动实践，以奋发有为的精神状态和真抓实干的实际行动，努力创造经得起历史和人民检验的实绩。学习贯彻习近平新时代中国特色社会主义思想主题教育中央第一指导组组长车俊，市委副书记、市长殷勇，市人大常委会主任李秀领，市政协主席魏小东，市委副书记刘伟，中央第一指导组副组长李小三参加。从治国理政新理念新思想新战略，到“十个明确”“十四个坚持”“十三个方面成就”，再到构建人类命运共同体等一系列中国主张，尹力引导大家深入思考“为什么学”。他指出，习近平新时代中国特色社会主义思想实现了马克思主义中国化时代化新的飞跃，我们要在“为什么学”上提高政治站位，持续增强学习的思想自觉和行动自觉。习近平新时代中国特色社会主义思想深刻回答了一系列重大时代课题，实现了对中国特色社会主义建设规律认识的新跃升；坚持以“两个结合”推进理论创新，标注了马克思主义发展的新高度；指引新时代实践取得历史性成就，展现出强大的真理力量和实践伟力；为全世界提供了更多中国智慧和中国方案，引领我国日益走近世界舞台中央。有这一思想的科学指引，我们就有了理论灯塔、思想旗帜、行动指南，就能够始终坚持正确方向，不断把党和人民事业发展推向新高度。“风雨不动安如山”般的理想信念、“江山就是人民、人民就是江山”的为民情怀……尹力向大家详细阐述了“学什么”。他指出，习近平新时代中国特色社会主义思想是博大精深、内涵丰富的科学理论体系，我们要在“学什么”上做到全面

In [27]:
#快速演示
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device=', device)

from transformers import BertModel

#加载预训练模型
pretrained = BertModel.from_pretrained('/mnt/mydisk/bert-base-chinese')
#需要移动到cuda上
pretrained.to(device)

#不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)


#定义下游任务模型
class Model(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                             attention_mask=attention_mask,
                             token_type_ids=token_type_ids)

        out = self.fc(out.last_hidden_state[:, 0])
        out = out.softmax(dim=1)
        return out


model = Model()
#同样要移动到cuda
model.to(device)

#虚拟一批数据,需要把所有的数据都移动到cuda上
input_ids = torch.ones(16, 100).long().to(device)
attention_mask = torch.ones(16, 100).long().to(device)
token_type_ids = torch.ones(16, 100).long().to(device)
labels = torch.ones(16).long().to(device)

#试算
model(input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids=token_type_ids).shape

#后面的计算和中文分类完全一样，只是放在了cuda上计算

device= cuda


Some weights of the model checkpoint at /mnt/mydisk/bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([16, 2])

# Start


In [28]:
import pandas as pd
import torch.utils.data as data

class CSVDataset(data.Dataset):
    def __init__(self, csv_file_path, text_column, label_column):
        self.data = pd.read_csv(csv_file_path)
        self.text_column = text_column
        self.label_column = label_column

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx][self.text_column]
        label = self.data.iloc[idx][self.label_column]
        return text, label
    

dataset = CSVDataset('/mnt/mydisk/CSTD/output.csv', 'title', 'label')
len(dataset)
for i in dataset:
    print(i)

('尹力在学习贯彻习近平新时代中国特色社会主义思想主题教育专题党课上强调 推动习近平新时代中国特色社会主义思想在京华大地落地生根形成生动实践 车俊殷勇李秀领魏小东刘伟李小三参加', 1)
('共青团北京市第十五次代表大会开幕 尹力讲话 贺军科殷勇魏小东刘伟出席', 1)
('尹力到市轨道交通指挥中心调研时强调 构建立体化现代化城市交通系统 更好满足城市发展和群众出行需求', 1)
('“五一”假期首日，尹力殷勇走进火车站、商圈检查安全生产和城市运行保障等工作，并慰问坚守岗位的劳动者！', 1)
('“五一”假期北京预计接待游客885万人次 各大热门公园景区基本约满', 1)
('市委常委会召开会议 听取全市社会建设工作情况汇报 研究“五一”安全防范和假日相关工作等事项 市委书记尹力主持会议', 1)
('市推进全国文化中心建设领导小组会议召开 尹力主持 殷勇出席', 1)
('尹力调研中轴线申遗保护工作时强调 让北京这座千年古都焕发出更加蓬勃的时代活力', 1)
('本市召开全市领导干部会议 尹力强调 深刻反思汲取教训 时刻保持警钟长鸣 全面开展全市安全生产隐患大排查大整治 殷勇刘伟出席', 1)
('平谷桃花节办了30余年 从带货大桃到带火文旅——桃谷花事', 1)
('市“两区”工作领导小组召开会议 尹力主持 殷勇刘伟出席', 1)
('尹力到市发展改革委、市财政局、市税务局、市统计局调研时强调 把主题教育学习成果转化为推动首都高质量发展实效', 1)
('市委常委会召开会议 传达学习习近平总书记重要讲话精神 研究部署全市深入开展主题教育工作等事项 市委书记尹力主持会议', 1)
('春季游热度攀升 旅游市场加速复苏', 1)
('市委城市工作委员会召开全体会议 尹力主持 殷勇刘伟出席', 1)
('书香浸古建 激活城市文化记忆', 1)
('博物馆之城 让金名片更加闪耀', 1)
('市委常委会召开会议 研究加快推动北京国际科技创新中心建设工作方案等事项 市委书记尹力主持会议', 1)
('“文化润疆”打造文化名片 《五星出东方》将在和田驻场演出', 1)
('尹力在金融工作座谈会上强调 贯彻落实党的二十大精神 推动首都金融高质量发展 易纲郭树清易会满殷勇参加', 1)
('三条文化带 承载古都“城市之魂”', 1)
('“中国发展高层论坛2023年年会”北京

In [29]:
from transformers import BertTokenizer

#加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-chinese')


def collate_fn(data):
    sents = [i[0] for i in data]
    labels = [i[1] for i in data]

    #编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=500,
                                   return_tensors='pt',
                                   return_length=True)

    #input_ids:编码之后的数字
    #attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)
    token_type_ids = data['token_type_ids'].to(device)
    labels = torch.LongTensor(labels).to(device)

    #print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels


#数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=4,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):
    break

print(len(loader))
input_ids.shape, attention_mask.shape, token_type_ids.shape, labels

6


(torch.Size([4, 500]),
 torch.Size([4, 500]),
 torch.Size([4, 500]),
 tensor([1, 1, 1, 1], device='cuda:0'))

In [30]:
from transformers import AdamW

#训练
optimizer = AdamW(model.parameters(), lr=5e-4)
criterion = torch.nn.CrossEntropyLoss()

model.train()
for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):
    out = model(input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)

    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if i % 5 == 0:
        out = out.argmax(dim=1)
        accuracy = (out == labels).sum().item() / len(labels)

        print(i, loss.item(), accuracy)

    if i == 100:
        break

0 0.723093569278717 0.25
5 0.41070568561553955 1.0


/mnt/mydisk/miniconda/envs/pytorch/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [12]:
model.eval()
correct = 0
total = 0

loader_test = torch.utils.data.DataLoader(dataset=Dataset('validation'),
                                            batch_size=32,
                                            collate_fn=collate_fn,
                                            shuffle=True,
                                            drop_last=True)

for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader_test):
        print(input_ids, attention_mask, token_type_ids,labels)

NameError: name 'Dataset' is not defined

In [8]:
#测试
def test():
    model.eval()
    correct = 0
    total = 0

    loader_test = torch.utils.data.DataLoader(dataset=Dataset('validation'),
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader_test):

        if i == 5:
            break

        print(i)

        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)

        out = out.argmax(dim=1)
        correct += (out == labels).sum().item()
        total += len(labels)

    print(correct / total)


test()

NameError: name 'Dataset' is not defined